## Story Generator Colab Server
https://colab.research.google.com/drive/1uNygzDR4hISwLOgmDS31hRHfr6KAF7Ib?usp=sharing

In [ ]:
!pip install -q transformers==4.35.2 accelerate bitsandbytes==0.41.1 huggingface_hub google-generativeai==0.3.1
!pip install -q Flask==2.3.2 Pillow==9.5.0 requests==2.30.0 flask-ngrok Flask-CORS==3.0.10 pyngrok
!pip install Flask-Cors
!pip install -q python-dotenv

from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Use environment variables
from huggingface_hub import login
login(os.getenv('HUGGINGFACE_API_KEY'))

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from IPython.display import clear_output
clear_output(wait=False)

print("Dependencies installed successfully!")

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import base64
from PIL import Image
import io
import google.generativeai as genai
import os
from google.colab import userdata

# Configure Gemini API
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-pro')

# Configure ngrok
ngrok.set_auth_token(userdata.get('NGROK_AUTH_TOKEN'))

app = Flask(__name__)
CORS(app)

@app.route('/generate_story', methods=['POST'])
def generate_story():
    try:
        data = request.json
        if not data:
            return jsonify({'success': False, 'error': 'No data received'}), 400
        
        genre = data.get('genre', 'fantasy')
        length = data.get('length', 'short')
        
        # Craft prompt for Gemini
        prompt = f"""Write a {genre.lower()} story with the following elements:
        - {length} length
        - Clear beginning, middle, and end
        - Age-appropriate content
        - Emotional resonance
        - Engaging plot and characters
        
        The story should be creative and original. Make it captivating for readers."""

        # Generate story using Gemini
        response = model.generate_content(prompt)
        
        if response.text:
            return jsonify({'success': True, 'story': response.text})
        else:
            return jsonify({'success': False, 'error': 'Failed to generate story'}), 500

    except Exception as e:
        print(f"Server error: {str(e)}")
        return jsonify({'success': False, 'error': f'Server error: {str(e)}'}), 500

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    api_url = f"{public_url.public_url}/generate_story"
    print(f'Server running at: {api_url}')
    
    app.run(port=5000)